In [2]:
import pandas as pd
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List, TypedDict
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
import re

# 定義狀態字典類型，用於保存每個客戶的狀態信息
class CustomerState(TypedDict):
    news_content: str  # 輸入的新聞內容
    customer_features: List[str]  # 客戶特徵列表
    extracted_features: str  # 提取的客戶特徵摘要
    news_summary: str  # 基於客戶特徵的新聞摘要
    investment_advice_result: str  # 生成的投資建議
    impact_score: int  # 新增的影響程度分數

# 定義個性化新聞代理類
class PersonalizedNewsAgent:

    def __init__(self, model: BaseChatModel, checkpointer: SqliteSaver):
        self.model = model  # 初始化生成式語言模型
        self.checkpointer = checkpointer  # 初始化檢查點保存器，用於保存狀態
        builder = StateGraph(CustomerState)  # 創建StateGraph並傳入狀態字典類型
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("extract_customer_features", self.extract_customer_features_node)
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)
        builder.add_node("investment_advice", self.investment_advice_node)
        builder.add_node("impact_assessment", self.impact_assessment_node)  # 新增影響評估節點
        builder.add_node("output_results", self.output_results_node)

        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "extract_customer_features")
        builder.add_edge("extract_customer_features", "personalized_news_summary")
        builder.add_edge("personalized_news_summary", "investment_advice")
        builder.add_edge("investment_advice", "impact_assessment")  # 添加新的邊
        builder.add_edge("impact_assessment", "output_results")
        self.graph = builder.compile(checkpointer=checkpointer)

    def initialize_node(self, state: CustomerState):
        return {
            "extracted_features": "",
            "news_summary": "",
            "investment_advice_result": "",
            "impact_score": 50,  # 初始化影響程度分數為 50（中等）
        }

    def extract_customer_features_node(self, state: CustomerState):
        customer_features_text = "\n".join(state["customer_features"])
        messages = [
            SystemMessage(content="以下所有產品皆為基金類型的產品，請直接列點總結每個客戶特徵，100字內，列點包括投資配置、產品偏好、風險偏好、投資策略、交易狀況、持倉狀況等，並確保生成內容完全來自以下客戶特徵信息，不參考其他來源。請用繁體中文回答。"),
            HumanMessage(content=customer_features_text),
        ]
        response = self.model.invoke(messages)
        state["extracted_features"] = response.content.strip()
        return {"extracted_features": state["extracted_features"]}

    def personalized_news_summary_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="請根據以下新聞內容及客戶特徵生成新聞重點摘要，個人化條列式生成3至5點，與客戶投資產品相關的新聞放在最前面，每點字數不超過30字，總字數少於200字，不要包括任何客戶特徵信息，只能從以下新聞內容生成摘要。請用繁體中文回答"),
            HumanMessage(content=state["news_content"])
        ]
        response = self.model.invoke(messages)
        state["news_summary"] = response.content.strip()
        return {"news_summary": state["news_summary"]}

    def investment_advice_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="你是一位具有 CFA Level 3 資格的資深投資專家，請嚴格基於每位客戶特徵和新聞摘要，列點式生成個人化投資建議，直接給具體建議即可。不要加入其他外部資訊，不要推薦任何產品，建議字數應在200字內。請確保新聞摘要中的資訊有確實反映在投資建議中，投資建議應該考慮到客戶的風險承受能力、投資目標、資產配置偏好，以及當前市場狀況，務必確保每個建議都是實際可行的。請用繁體中文回答。"),
            AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
            AIMessage(content=f"新聞摘要: {state['news_summary']}"),
        ]
        response = self.model.invoke(messages)
        state["investment_advice_result"] = response.content.strip()
        return {"investment_advice_result": state["investment_advice_result"]}

    def impact_assessment_node(self, state: CustomerState):
        messages = [
        SystemMessage(content="你是一位具有 CFA Level 3 資格的資深投資專家，請評估新聞對每個客戶的影響程度。給出0到100分的評分，分數越高代表影響越大。"),
        AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
        AIMessage(content=f"新聞摘要: {state['news_summary']}"),
    ]
        response = self.model.invoke(messages)
        impact_text = response.content.strip()

    # 初始化默認值
        score = 20
        explanation = "影響甚小，先給予20分"

    # 檢查是否模型返回了完整的提示而不是结果
        if "0到100分的評分" not in impact_text:
        # 使用正則表達式提取数字評分
            score_match = re.search(r'\b(\d{1,3})\b', impact_text)
            if score_match:
                score = int(score_match.group(1))
            # 保证分數在0到100之间
                score = min(max(score, 0), 100)
                explanation = impact_text[impact_text.find(score_match.group(1)) + len(score_match.group(1)):].strip()
                explanation = explanation.replace('*', '').strip()  # 移除可能的米字號或其他不必要符号
            else:

            # 如果没有提取到有效分数，使用範例值並解釋
                score = 20
                explanation = "影響甚小，先給予20分"

    # 构建影响评分结果，确保没有多余的分数后缀
        state["impact_score"] = f"{score} 分" if explanation else f"{score} 分 - {explanation}"
        return {"impact_score": state["impact_score"]}

    def output_results_node(self, state: CustomerState):
        return {
            "extracted_features": state["extracted_features"],
            "news_summary": state["news_summary"],
            "investment_advice_result": state["investment_advice_result"],
            "impact_score": state["impact_score"],  # 包含影響程度分數
        }

# 定義格式化輸出結果的函數
def format_output_for_customer(customer_name, extracted_features, news_summary, investment_advice_result, impact_score):
    return (f"客戶名稱: {customer_name}\n"
            f"提取的客戶特徵:\n{extracted_features}\n\n"
            f"新聞摘要:\n{news_summary}\n\n"
            f"投資建議:\n{investment_advice_result}\n\n"
            f"新聞對客戶的影響程度: {impact_score} 分\n")

# 初始化模型和檢查點保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
memory = SqliteSaver.from_conn_string(":memory:")

bot = PersonalizedNewsAgent(model=model, checkpointer=memory)
    
thread = {"configurable": {"thread_id": "1"}}

# 讀取Excel表格
df = pd.read_excel("teste.xlsx")
    
# 定義輸入的新聞內容
state = {
    "news_content": """投資策略 
                       -增持金融債，增加存續期間：過去在聯準會啟動降息後，殖利率皆呈現一路下行趨勢。此外，在預防性降息階段，像是金融債等景氣敏感產業，將受惠利差收斂。
                       -少量調節科技股，等待修正後的伺機加碼機會：時序步入9月，股市可能向下再打第二隻腳，幅度上或許不會如同7~8月的下跌波段劇烈，但手上科技部位仍可少量減碼。
                       -公用事業可逐步獲利了結：7月以來，公用事業發揮了很好的避險效果，在市場動盪之際，仍繳出超過雙位數的報酬表現，隨著市場可能再度震盪，可適度獲利了結。
                       市場展望
                       -匯市展望：BoJ總裁鷹派發言，帶動日圓重新轉強，疊加市場過度樂觀聯準會降息空間，導致美元指數近期的疲態。短期美債殖利率再下跌空間有限，將給予美元指數提供支撐。
                       -債市展望：當前十年期殖利率與政策利率兩者差距約在-1.7%附近，處於歷史低位，此現象凸顯出，目前殖利率市場似乎過度樂觀降息，預期短期在9/18聯準會尚未實際啟動降息前，兩者差距再擴大空間有限。短期十年期殖利率區間預估3.7~4.1%。
                       -股市展望：歷年9月為美股走勢相對較差的月份，此外，今年剛好恰逢總統大選，根據過去經驗，總統大選年度將加深9月份的市場波動，因此短線上仍須留意。所幸，中期有預防性降息支撐，過去三次預防性降息皆為股市後續提供正面支撐。
                       頭條新聞
                       -利比亞將暫停原油生產。利比亞原油產量達118萬桶/日，占全球原油供給1.1%，受到央行總裁被首都政府單方面撤換，掌控國內大部分油田的東政府，宣稱將暫停原油生產、出口，一度激勵油價短期漲勢。
                       -Kennedy宣布退選，並支持共和黨川普。作為甘迺迪前總統侄子的獨立參選人-Robert F. Kennedy Jr.於8/23正式宣布退出本屆美國總統大選，並轉為支持川普。根據7大搖擺州的民調， Kennedy擁有3.6~5.6%支持度。
                       -植田和男表示，BoJ仍在升息的道路上。BoJ總裁在國會聽證會上表示，只要經濟成長和通膨符合預期，日銀仍將在升息的道路上前行，下次升息時點或許比原預期更久，但不會因為市場動盪就不考慮升息可能。
                       產業訊息
                       -NVIDIA 毛利率不及預期， 盤後股價重挫。Nvidia 2Q25營收300億美元，年增122%，EPS 0.68美元，分別高於市場預估的287億、0.64美元。然而，3Q25營收、毛利率指引不及預期，引發失望性賣壓。
                        AI伺服器兩大代工廠對需求持正面展望。作為AI伺服器主要代工廠之一的廣達董事長-林百里，雖未正面回應Nvidia晶片出貨延宕問題，但表示廣達AI需求沒問題，同時近期高盛拜訪鴻海，也獲得管理層預期，年底AI伺服器將持續實現月成長。
                       -蘋果將於9/10舉行秋季發表會。蘋果已正式寄發邀請函，將於9/10舉行秋季發表會，預期將推出撘載最新AI功能的iPhone 16、Apple Watch、AirPods等新品。然而，郭明錤認為， iPhone 16 僅撘載測試版的Apple
                        Intelligence，以及Apple Intelligence Siri僅支援英語，難以激勵訂單出現太大成長。
                       總體經濟
                       -7月台灣景氣對策燈號由6月的38分(紅燈)轉為35分(黃紅燈)，主要是受到凱米颱風影響部分生產及出貨遞延，導致工業生產指數由15.8分(紅燈)下滑至10.2分(黃紅燈)、出口值由26.9分(紅燈)下滑至8.1分(綠燈)。
                       -以台灣出口動能觀察，7月出口金額399億新台幣(年增3.1%)，延續近3個月的上升趨勢，而去年下半年平均出口金額為383億，因此只要後續出口動能維持穩定的季節性脈動，燈號細項的海關出口值在未來應該能有效回穩。
                       -參考過去經驗，當第一顆紅燈亮起後，只要景氣燈號能夠維持在黃紅燈~紅燈區間，未來6個月股市平均報酬率仍有9.68%(出現過3次，勝率100%)，因此不用擔憂這次紅燈轉向黃紅燈的變化。
                      外匯市場
                      -近月美元指數下跌幅度超過3%，創下2022/11以來的單月下跌新高紀錄，反應市場過度擔憂經濟衰退。目前美元指數季線乖離率-3.4%，短期有望走出反彈格局。
                      -受到利比亞可能暫停原油生產、出口影響，短期一度激勵油價走勢。然而，10月OPEC+可能逐步恢復自願性減產220萬桶/日，將抵銷利比亞出口減少的衝擊，需求放緩仍是油價上方面臨的壓力主因。
                      -地緣政治風險，疊加聯準會降息預期，推升近月金價走勢，金價持續向上突破新高。然而，隨著短期利率再下行空間有限，金價應維持高檔震盪。
                      -隨著勞動市場數據表現不佳，美元指數自6月高檔106價位，一路下跌，加上市場預期ECB降息步伐將慢於聯準會(年內3碼，明年上半年2碼)，美德利差下行，美元指數甚至向下挑戰100關鍵防守價位。
                      -然而，美元指數與美債殖利率的下跌，似乎過度反應衰退風險與降息預期。首先在勞動市場方面，被解聘者占總失業人數比例為48.7%，與今年1~6月平均48.6%相比，上升幅度不大。因此，現在就要判定聯準會明年將一路降息，仍言之過早。
                      -此外，觀察美元期貨淨部位市場，投機者看多倉位並未隨著這波美元指數的下跌，而有所下滑，顯示市場參與者認為，美元指數將迎來短期反彈。
                      商品市場
                      -利比亞央行行長-薩迪克.卡比爾，被首都政權單方面解任，導致東部政權宣布將關閉油田(掌控國家大部分油田)，供給端可能受到衝擊的影響，激勵國際油價大漲。
                      -利比亞原油產量落在118萬/日，為OPEC第7大產油國，占全球原油供給規模1.1~1.2%，並不算特別重要的生產國。按照國際三大能源機構對於明年供給的預估觀察，2025年全球原油供給將增加190萬桶/日(OPEC+可能於10月後逐步恢復220萬/日的自願減產)，預期能抵銷產量減少的衝擊。
                      -因此，就算利比亞國內政治局勢持續混亂，原油暫停出口時間延長，對於國際原油供需衝擊有限，油價在短期事件衝擊後，將重新回歸需求端的檢視。
                      債市市場
                      -上週十年期殖利率上彈至3.9%，在利差方面，8/5後美國投等債利差由110bps高檔，一路向下收斂至93bps，成為支撐投等債表現關鍵。
                      -上週非投等債全面上漲，利差持續向下收斂，尤其CCC及以下評等由9.45%降至9.39%，降幅最為明顯，顯示市場情緒正在改善。
                      -上週新當債下跌0.73%，主要受到中國與墨西哥殖利率的上彈。8月人行並未調降MLF利率，金額方面，僅展開3,000億續作，低於到期的4,010億人民幣，十年期殖利率由低檔向上反彈。在墨西哥部分，儘管月初央行以3:2
                         的票數降息1碼至10.75%，然而連續5個月上升的通膨，仍給予央行後續決策壓力，墨西哥十年期殖利率在8/9降息後，反而逆向上漲。
                      -隨著鮑威爾於Jackson Hole演講中，明確釋出降息訊號，9月聯準會高機率啟動預防性降息，而降息對於循環性產業的基本面動能，將起到托底作用。
                      -參考前一次預防性降息2019/7~10情形(2020應對疫情衝擊降息，並不被納入考量)，在啟動降息期間，以科技、金融、原材料產業的債券表現較為優異，OAS分別下滑6bps、3.2bps、3bps。
                      -因此儘管自6月開始， 投等債利差已連續3個月向上反彈(經濟衰退疑慮、9月為歷年發債高峰)，但預期在降息支撐下，像是目前主推的金融債仍將受惠。
                      -觀察過去走勢，十年期殖利率長時間維持在政策利率上方，唯有當聯準會將利率升至高峰，準備調降利率階段，才會見到十年期殖利率衰先下行，跌至政策利率下方。
                      -截至今年8月，隨著美國十年期殖利率一度下跌3.78%，其與Fed政策利率差值創下2023/5以來的新高，達到-1.7%附近的歷史相對低位(最低值為2023/5/3所創下的-1.91%)。
                      -此現象凸顯出，目前殖利率市場似乎過度樂觀降息，或對未來景氣過於悲觀，預期短期在9/18聯準會尚未實際啟動降息前，兩者差距再擴大空間有限(除非發生意外衝擊，大幅強化急速降息預期)。
                      股票市場
                      -8/5全球股市重挫後，半導體、科技股迎來反彈行情，然而近期動能開始放緩，上週科技股延續跌勢，加上短期Nvidia財報不如預期，短期震盪可能延續，資金更為偏好價值股。
                      -經濟數據不佳，疊加人行8月並未額外釋放流動性，滬深300延續跌勢，距離2月3,108低點約5%。然而金融股已領先大盤止穩，四大行股價甚至創下2018年以來新高，陸股距離底部或許已經不遠。
                      -上週印度為少數能維持上漲的國家，MSCI權重的上調持續帶給印度股市正面影響。自8/13 MSCI公告權重調整後，外資由8月原先的淨流出17億美元，轉為淨流入11億美元(截至8/27)。
                      -川普與賀錦麗雙方皆同意於9/10展開第二次的總統大選辯論，雙方將再次針對經濟、外交等議題進行爭鋒。賀錦麗已明確提出將企業稅由21%提高至28%，與川普希望的再次降稅，形成鮮明對比，預期在經濟政策
                       方面，稅制問題將再次成為攻防重點。
                      -歷年9月是美股普遍表現較差的月份，過去30年下跌機率達50%，平均跌幅-0.81%，而在總統選舉年度，由於總統辯論集中在9月舉行，選情變化牽動市場情緒，下跌機率上升至57.1%，平均跌幅也升高至1.38%。
                      -因此儘管8月中下旬，市場在跌深後出現衰退預期和緩的反彈，但9月份市場仍有變數，尤其近期賀錦麗在搖擺州聲勢明顯升高，今年由誰問鼎白宮仍是未知數。
                      -鮑威爾在全球央行年會上，明確釋出降息時機已經到來，9月啟動降息基本上已大勢底定。
                      -過去聯準會一共有3次在經濟實際步入衰退前的預防性降息經驗，除了1995年是在降息後，股市便一路上漲，其餘兩次，1998、2019年都呈現降息初期先下跌(10~15個交易日)，後續才迎來中期上漲的格局。
                      -也因此從三次平均的角度而言， 降息後1 個月，S&P500報酬率為-0.1%，後3個月為7.2%(僅2019年是負報酬)，後6個月為15.2%，降息將提供股市流動性與估值上的支撐。
                      -中國滬深300指數今年反彈至5月下旬3,700點後，已連續三個月走低，持續反應弱於預期的消費動能和企業信心。以當前3,300點而言，距離跌破2月3,108低點剩下5%空間。
                      -然而在過去一個月，儘管大盤下跌2.5%，但最大權值的金融股卻逆勢上漲3%，指標股-國有四大行股價更是創2018年以來的新高，反應銀行業績在2Q24見到止穩(淨息差維持1.54%低檔，未再創低)，同時官方、保險資金也持續湧入，試圖給予盤面支撐。
                      -2012年以來，在下跌階段，金融股一共出現3次提前大盤落底的經驗，分別是2012/9/21~2012/11/30、2018/8/17~2018/12/28、2024/1/2~2024/2/2，而當前金融股又再次提前落底，疊加成交量大幅萎縮、估值便宜，或暗示中國股市即將落底。
                  """  
}

# 生成客戶特徵
all_customers_output = []
all_customers_data = []

# 你可以選擇指定要使用的欄位名稱
selected_columns = ['客戶風險等級','近一年總成交量', '股票類型交易量', '債券類型交易量', '平衡類型交易量', '股票類型交易量佔比', '債券類型交易量佔比', '平衡類型交易量佔比'
                    ,'近一年總成交次數','股票類型交易次數','債券類型交易次數','平衡類型交易次數','股票類型交易次數佔比','債券類型交易次數佔比'
                    ,'平衡類型交易次數佔比','庫存量','股票類型庫存量','債券類型庫存量','平衡類型庫存量','股票類型庫存量佔比','債券類型庫存量佔比'
                    ,'平衡類型庫存量佔比','RR1佔比','RR2佔比','RR3佔比','RR4佔比','RR5佔比','近一年REITs的總成交量','近一年大中華股票的總成交量'
                    ,'近一年中國股票的總成交量','近一年巴西股票的總成交量','近一年日本中小型股票的總成交量','近一年日本股票的總成交量'
                    ,'近一年台灣股票的總成交量','近一年全球平衡的總成交量','近一年全球股票的總成交量','近一年全球債的總成交量','近一年印度股票的總成交量'
                    ,'近一年亞太股票的總成交量','近一年亞太區(除日本)股票的總成交量','近一年亞太區股票的總成交量','近一年亞太區貨幣債的總成交量'
                    ,'近一年拉美股票的總成交量','近一年東南亞股票的總成交量','近一年金融股票 的總成交量','近一年南韓股票的總成交量'
                    ,'近一年科技股票的總成交量','近一年美元高息/高收益債券的總成交量','近一年美國股票的總成交量','近一年美債的總成交量'
                    ,'近一年英國股票的總成交量','近一年香港股票的總成交量','RR5佔比','近一年高收益債的總成交量','近一年貨幣型的總成交量'
                    ,'近一年黃金的總成交量','近一年新興市場股票的總成交量','近一年新興市場債的總成交量','近一年新興歐洲股票的總成交量'
                    ,'近一年德國股票的總成交量','近一年歐洲中小型股票的總成交量','近一年歐洲股票的總成交量','近一年歐洲新興市場股票的總成交量'
                    ,'近一年環球股票的總成交量','近一年環球高息/高收益債券的總成交量','近一年環球債券的總成交量','近一年環球新市債的總成交量'
                    ,'近一年環球新興市場股票的總成交量','近一年環球新興市場強勢貨幣債券的總成交量','近一年醫療生技的總成交量'
                    ,'近一年醫療保健股票的總成交量','近一年礦業股票的總成交量','近一年REITs的總成交量佔比','近一年大中華股票的總成交量佔比'
                    ,'近一年中國股票的總成交量佔比','近一年巴西股票的總成交量佔比','近一年日本中小型股票的總成交量佔比'
                    ,'近一年日本股票的總成交量佔比','近一年台灣股票的總成交量佔比','近一年全球平衡的總成交量佔比','近一年全球股票的總成交量佔比'
                    ,'近一年全球債的總成交量佔比','近一年印度股票的總成交量佔比','近一年亞太股票的總成交量佔比','近一年亞太區(除日本)股票的總成交量佔比'
                    ,'近一年亞太區股票的總成交量佔比','近一年亞太區貨幣債的總成交量佔比','近一年拉美股票的總成交量佔比','近一年東南亞股票的總成交量佔比'
                    ,'近一年金融股票 的總成交量佔比','近一年南韓股票的總成交量佔比','近一年科技股票的總成交量佔比','近一年美元高息/高收益債券的總成交量佔比'
                    ,'近一年美國股票的總成交量佔比','近一年美債的總成交量佔比','近一年英國股票的總成交量佔比','近一年香港股票的總成交量佔比'
                    ,'近一年能源股票 的總成交量佔比','近一年高收益債的總成交量佔比','近一年貨幣型的總成交量佔比','近一年黃金的總成交量佔比'
                    ,'近一年新興市場股票的總成交量佔比','近一年新興市場債的總成交量佔比','近一年新興歐洲股票的總成交量佔比','近一年德國股票的總成交量佔比'
                    ,'近一年歐洲中小型股票的總成交量佔比','近一年歐洲股票的總成交量佔比','近一年歐洲新興市場股票的總成交量佔比','近一年環球股票的總成交量佔比'
                    ,'近一年環球高息/高收益債券的總成交量佔比','近一年環球債券的總成交量佔比','近一年環球新市債的總成交量佔比'
                    ,'近一年環球新興市場股票的總成交量佔比','近一年環球新興市場強勢貨幣債券的總成交量佔比','近一年醫療生技的總成交量佔比'
                    ,'近一年醫療保健股票的總成交量佔比','近一年礦業股票的總成交量佔比','近一年REITs的總成交次數','近一年大中華股票的總成交次數'
                    ,'近一年中國股票的總成交次數','近一年巴西股票的總成交次數','近一年日本中小型股票的總成交次數','近一年日本股票的總成交次數'
                    ,'近一年台灣股票的總成交次數','近一年全球平衡的總成交次數','近一年全球股票的總成交次數','近一年全球債的總成交次數'
                    ,'近一年印度股票的總成交次數','近一年亞太股票的總成交次數','近一年亞太區(除日本)股票的總成交次數','近一年亞太區股票的總成交次數'
                    ,'近一年亞太區貨幣債的總成交次數','近一年拉美股票的總成交次數','近一年東南亞股票的總成交次數','近一年金融股票 的總成交次數'
                    ,'近一年南韓股票的總成交次數','近一年科技股票的總成交次數','近一年美元高息/高收益債券的總成交次數','近一年美國股票的總成交次數'
                    ,'近一年美債的總成交次數','近一年英國股票的總成交次數','近一年香港股票的總成交次數','近一年能源股票 的總成交次數'
                    ,'近一年高收益債的總成交次數','近一年貨幣型的總成交次數','近一年黃金的總成交次數','近一年新興市場股票的總成交次數'
                    ,'近一年新興市場債的總成交次數','近一年新興歐洲股票的總成交次數','近一年德國股票的總成交次數','近一年歐洲中小型股票的總成交次數'
                    ,'近一年歐洲股票的總成交次數','近一年歐洲新興市場股票的總成交次數','近一年環球股票的總成交次數','近一年環球高息/高收益債券的總成交次數'
                    ,'近一年環球債券的總成交次數','近一年環球新市債的總成交次數','近一年環球新興市場股票的總成交次數','近一年環球新興市場強勢貨幣債券的總成交次數'
                    ,'近一年醫療生技的總成交次數','近一年醫療保健股票的總成交次數','近一年礦業股票的總成交次數','近一年REITs的總成交次數佔比'
                    ,'近一年大中華股票的總成交次數佔比','近一年中國股票的總成交次數佔比','近一年巴西股票的總成交次數佔比','近一年日本中小型股票的總成交次數佔比'
                    ,'近一年日本股票的總成交次數佔比','近一年台灣股票的總成交次數佔比','近一年全球平衡的總成交次數佔比','近一年全球股票的總成交次數佔比'
                    ,'近一年全球債的總成交次數佔比','近一年印度股票的總成交次數佔比','近一年亞太股票的總成交次數佔比','近一年亞太區(除日本)股票的總成交次數佔比'
                    ,'近一年亞太區股票的總成交次數佔比','近一年亞太區貨幣債的總成交次數佔比','近一年拉美股票的總成交次數佔比','近一年東南亞股票的總成交次數佔比'
                    ,'近一年金融股票 的總成交次數佔比','近一年南韓股票的總成交次數佔比','近一年科技股票的總成交次數佔比'
                    ,'近一年美元高息/高收益債券的總成交次數佔比','近一年美國股票的總成交次數佔比','近一年美債的總成交次數佔比'
                    ,'近一年英國股票的總成交次數佔比','近一年香港股票的總成交次數佔比','近一年能源股票 的總成交次數佔比','近一年高收益債的總成交次數佔比'
                    ,'近一年貨幣型的總成交次數佔比','近一年黃金的總成交次數佔比','近一年新興市場股票的總成交次數佔比','近一年新興市場債的總成交次數佔比'
                    ,'近一年新興歐洲股票的總成交次數佔比','近一年德國股票的總成交次數佔比','近一年歐洲中小型股票的總成交次數佔比','近一年歐洲股票的總成交次數佔比'
                    ,'近一年歐洲新興市場股票的總成交次數佔比','近一年環球股票的總成交次數佔比','近一年環球高息/高收益債券的總成交次數佔比'
                    ,'近一年環球債券的總成交次數佔比','近一年環球新市債的總成交次數佔比','近一年環球新興市場股票的總成交次數佔比'
                    ,'近一年環球新興市場強勢貨幣債券的總成交次數佔比','近一年醫療生技的總成交次數佔比','近一年醫療保健股票的總成交次數佔比'
                    ,'近一年礦業股票的總成交次數佔比','REITs的庫存量','大中華股票的庫存量','中國股票的庫存量','巴西股票的庫存量'
                    ,'日本中小型股票的庫存量','日本股票的庫存量','台灣股票的庫存量','全球平衡的庫存量','全球股票的庫存量','全球債的庫存量'
                    ,'印度股票的庫存量','亞太股票的庫存量','亞太區(除日本)股票的庫存量','亞太區股票的庫存量','亞太區貨幣債的庫存量','拉美股票的庫存量'
                    ,'東南亞股票的庫存量','金融股票 的庫存量','南韓股票的庫存量','科技股票的庫存量','美元高息/高收益債券的庫存量','美國股票的庫存量'
                    ,'美債的庫存量','英國股票的庫存量','香港股票的庫存量','能源股票 的庫存量','高收益債的庫存量','貨幣型的庫存量','黃金的庫存量'
                    ,'新興市場股票的庫存量','新興市場債的庫存量','新興歐洲股票的庫存量','德國股票的庫存量','歐洲中小型股票的庫存量','歐洲股票的庫存量'
                    ,'歐洲新興市場股票的庫存量','環球股票的庫存量','環球高息/高收益債券的庫存量','環球債券的庫存量','環球新市債的庫存量'
                    ,'環球新興市場股票的庫存量','環球新興市場強勢貨幣債券的庫存量','醫療生技的庫存量','醫療保健股票的庫存量','礦業股票的庫存量'
                    ,'REITs的庫存量佔比','大中華股票的庫存量佔比','中國股票的庫存量佔比','巴西股票的庫存量佔比','日本中小型股票的庫存量佔比'
                    ,'日本股票的庫存量佔比','台灣股票的庫存量佔比','全球平衡的庫存量佔比','全球股票的庫存量佔比','全球債的庫存量佔比','印度股票的庫存量佔比'
                    ,'亞太股票的庫存量佔比','亞太區(除日本)股票的庫存量佔比','亞太區股票的庫存量佔比','亞太區貨幣債的庫存量佔比','拉美股票的庫存量佔比'
                    ,'東南亞股票的庫存量佔比','金融股票 的庫存量佔比','南韓股票的庫存量佔比','科技股票的庫存量佔比','美元高息/高收益債券的庫存量佔比'
                    ,'美國股票的庫存量佔比','美債的庫存量佔比','英國股票的庫存量佔比','香港股票的庫存量佔比','能源股票 的庫存量佔比','高收益債的庫存量佔比'
                    ,'貨幣型的庫存量佔比','黃金的庫存量佔比','新興市場股票的庫存量佔比','新興市場債的庫存量佔比','新興歐洲股票的庫存量佔比'
                    ,'德國股票的庫存量佔比','歐洲中小型股票的庫存量佔比','歐洲股票的庫存量佔比','歐洲新興市場股票的庫存量佔比','環球股票的庫存量佔比'
                    ,'環球高息/高收益債券的庫存量佔比','環球債券的庫存量佔比','環球新市債的庫存量佔比','環球新興市場股票的庫存量佔比'
                    ,'環球新興市場強勢貨幣債券的庫存量佔比','醫療生技的庫存量佔比','醫療保健股票的庫存量佔比','礦業股票的庫存量佔比'
                   ]

# 迭代每個客戶，生成特徵摘要和投資建議
for index, row in df.iterrows():
    customer_name = row['ID']
    
    # 根據選擇的欄位來生成客戶特徵
    customer_features = []
    for col in selected_columns:
        if pd.notna(row[col]):  # 檢查該欄位是否有數值
            feature = f"{col}: {row[col]}"
            customer_features.append(feature)
    
    # 更新狀態
    state["customer_features"] = customer_features

    # 執行圖形流程
    result = bot.graph.invoke(state, thread)

    # 檢查生成的投資建議是否為空，如果是空的，給予警告並記錄
    if not result["investment_advice_result"]:
        print(f"警告: {customer_name} 的投資建議生成為空！")

    # 格式化輸出結果
    customer_output = format_output_for_customer(
        customer_name, 
        result["extracted_features"], 
        result["news_summary"],
        result["investment_advice_result"], 
        result["impact_score"]  # 添加影響程度分數
    )
    
    all_customers_output.append(customer_output)

    all_customers_data.append({
        "客戶": customer_name,
        "客戶特徵": result["extracted_features"],
        "新聞摘要": result["news_summary"],
        "投資建議": result["investment_advice_result"],
        "影響程度分數": result["impact_score"]  # 新增的影響程度分數列
    })

# 創建DataFrame並輸出到Excel
output_df = pd.DataFrame(all_customers_data)
with pd.ExcelWriter("Personalized_Investment_Advice_Output.xlsx", engine="openpyxl") as writer:
    output_df.to_excel(writer, sheet_name="Sheet1", index=False)
    worksheet = writer.sheets["Sheet1"]
    
    # 調整欄位寬度
    for i, col in enumerate(output_df.columns):
        max_width = max(output_df[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.column_dimensions[get_column_letter(i + 1)].width = max_width
    
    # 文字換行
    for row in worksheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

print("Excel 文件已生成並保存，請檢查結果。")

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\senpai-m2n-YTg9-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Excel 文件已生成並保存，請檢查結果。


In [32]:
import nltk
nltk.data.path = ['C:\\Users\\user\\AppData\\Roaming\\nltk_data']  # 手動指定 NLTK 資料的路徑
nltk.download('punkt', download_dir='C:\\Users\\user\\AppData\\Roaming\\nltk_data')  # 強制下載 punkt 模型

import pdfplumber
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer  # LSA 算法，用來生成摘要
import pandas as pd

# Step 1: 提取單頁 PDF 文本
def extract_text_from_page(pdf, page_number):
    page = pdf.pages[page_number]
    return page.extract_text()

# Step 2: 使用 sumy 進行單頁摘要
def summarize_text(text, sentence_count=5):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentence_count)
    return " ".join([str(sentence) for sentence in summary])

# Step 3: 結合提取文本和摘要，並寫入 Excel
def extract_summarize_and_save_to_excel(pdf_path, sentence_count=5, excel_path="summary_output.xlsx"):
    with pdfplumber.open(pdf_path) as pdf:
        page_summaries = []
        
        # 逐頁處理
        for page_number in range(len(pdf.pages)):
            pdf_text = extract_text_from_page(pdf, page_number)
            
            if pdf_text:
                summary = summarize_text(pdf_text, sentence_count)
                page_summaries.append((page_number + 1, summary))
            else:
                page_summaries.append((page_number + 1, "無法提取文本"))

        # 將摘要寫入到 DataFrame
        df_summary = pd.DataFrame(page_summaries, columns=["Page Number", "Summary"])
        
        # 將 DataFrame 寫入 Excel 文件
        df_summary.to_excel(excel_path, index=False)
        
        return f"摘要已成功保存到 {excel_path}"

# 示例使用
pdf_path = "test.pdf"  # 替換為你的 PDF 路徑
excel_output_path = "summary_output.xlsx"  # 要保存的 Excel 文件路徑
result_message = extract_summarize_and_save_to_excel(pdf_path, sentence_count=5, excel_path=excel_output_path)
print(result_message)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


摘要已成功保存到 summary_output.xlsx


In [42]:
import pdfplumber
import pytesseract
import cv2
import pandas as pd
from PIL import Image
import numpy as np
from io import BytesIO
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph
from typing import List, TypedDict
import re  # 用於正則表達式處理

# 初始化 OCR
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # Windows 的 tesseract 路徑

# 定義狀態字典類型，用於保存每個客戶的狀態信息
class CustomerState(TypedDict):
    news_content: str  # 輸入的新聞內容
    customer_features: List[str]  # 客戶特徵列表
    extracted_features: str  # 提取的客戶特徵摘要
    news_summary: str  # 基於客戶特徵的新聞摘要
    investment_advice_result: str  # 生成的投資建議
    impact_score: int  # 新增的影響程度分數

# 定義個性化新聞代理類
class PersonalizedNewsAgent:

    def __init__(self, model):
        self.model = model
        self.checkpointer = SqliteSaver.from_conn_string(":memory:")
        builder = StateGraph(CustomerState)
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("extract_customer_features", self.extract_customer_features_node)
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)
        builder.add_node("investment_advice", self.investment_advice_node)
        builder.add_node("impact_assessment", self.impact_assessment_node)
        builder.add_node("output_results", self.output_results_node)
        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "extract_customer_features")
        builder.add_edge("extract_customer_features", "personalized_news_summary")
        builder.add_edge("personalized_news_summary", "investment_advice")
        builder.add_edge("investment_advice", "impact_assessment")
        builder.add_edge("impact_assessment", "output_results")
        self.graph = builder.compile(checkpointer=self.checkpointer)

    def initialize_node(self, state: CustomerState):
        return {
            "extracted_features": "",
            "news_summary": "",
            "investment_advice_result": "",
            "impact_score": 50,  # 初始化影響程度分數為 50（中等）
        }

    def extract_customer_features_node(self, state: CustomerState):
        customer_features_text = "\n".join(state["customer_features"])
        messages = [
            {"role": "system", "content": "請總結每個客戶的特徵，並且只使用提供的客戶特徵生成摘要。"},
            {"role": "user", "content": customer_features_text}
        ]
        response = self.model.invoke(messages)
        state["extracted_features"] = response.content.strip()
        return {"extracted_features": state["extracted_features"]}

    def personalized_news_summary_node(self, state: CustomerState):
        # 將提取的新聞內容傳遞給 ChatGoogleGenerativeAI 模型
        messages = [
            {"role": "system", "content": "根據以下新聞內容生成摘要，並與客戶特徵相關。"},
            {"role": "user", "content": state["news_content"]}
        ]
        response = self.model.invoke(messages)
        state["news_summary"] = response.content.strip()
        return {"news_summary": state["news_summary"]}

    def investment_advice_node(self, state: CustomerState):
        messages = [
            {"role": "system", "content": "請根據新聞摘要與客戶特徵生成投資建議，字數在200字內。"},
            {"role": "user", "content": f"客戶特徵: {state['extracted_features']}\n新聞摘要: {state['news_summary']}"}
        ]
        response = self.model.invoke(messages)
        state["investment_advice_result"] = response.content.strip()
        return {"investment_advice_result": state["investment_advice_result"]}

    def impact_assessment_node(self, state: CustomerState):
        messages = [
            {"role": "system", "content": "請評估新聞對客戶的影響程度，0到100分，越高影響越大。"},
            {"role": "user", "content": f"客戶特徵: {state['extracted_features']}\n新聞摘要: {state['news_summary']}"}
        ]
        response = self.model.invoke(messages)
        impact_text = response.content.strip()

        score = 20  # 默認值
        score_match = re.search(r'\b(\d{1,3})\b', impact_text)
        if score_match:
            score = int(score_match.group(1))
            score = min(max(score, 0), 100)  # 保證分數在0到100之間
        state["impact_score"] = score
        return {"impact_score": state["impact_score"]}

    def output_results_node(self, state: CustomerState):
        return {
            "extracted_features": state["extracted_features"],
            "news_summary": state["news_summary"],
            "investment_advice_result": state["investment_advice_result"],
            "impact_score": state["impact_score"],
        }

# 初始化 ChatGoogleGenerativeAI 模型
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
bot = PersonalizedNewsAgent(model=model)

# Step 1: 自動讀取 PDF 中每頁的文本並保存到 news_content
pdf_path = 'test.pdf'  # 替換為您的 PDF 文件路徑
news_contents = []

def extract_text_from_image(image):
    """使用 Tesseract OCR 從圖像中提取文本"""
    # 將圖像轉為灰階
    gray_image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    # 提取文本
    text = pytesseract.image_to_string(gray_image)
    return text.strip()

with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        # 提取文本
        text = page.extract_text()
        # 提取圖像並處理
        image_texts = []
        for image_object in page.images:
            # 提取原始圖像數據
            x0, y0, x1, y1 = image_object["x0"], image_object["top"], image_object["x1"], image_object["bottom"]
            cropped_image = page.within_bbox((x0, y0, x1, y1)).to_image().original
            image = Image.open(BytesIO(cropped_image))  # 這裡的 BytesIO 將字節流轉換為 PIL 圖像
            extracted_text = extract_text_from_image(image)  # 使用 OCR 提取圖像中的文本
            image_texts.append(extracted_text)
        
        combined_text = text or ""  # 如果文本為 None，使用空字串
        combined_text += "\n".join(image_texts)  # 添加來自圖像的文本
        news_contents.append({"page_number": i+1, "content": combined_text.strip()})

# 確保我們提取到的文本是有效的
if not news_contents:
    raise ValueError("無法提取 PDF 內容，請檢查 PDF 文件。")

# 使用模型生成每頁的新聞摘要
pdf_summary_data = []
for page in news_contents:
    state = {
        "news_content": page["content"]  # 傳遞每頁的內容
    }
    result = bot.graph.invoke(state)
    pdf_summary_data.append({
        "Page Number": page["page_number"],
        "News Summary": result["news_summary"]
    })

# 將每頁的摘要寫入 Excel
pdf_summary_df = pd.DataFrame(pdf_summary_data)
pdf_summary_excel_path = "PDF_Summary_with_Summaries.xlsx"

with pd.ExcelWriter(pdf_summary_excel_path, engine="openpyxl") as writer:
    pdf_summary_df.to_excel(writer, sheet_name="PDF Summary", index=False)
    worksheet = writer.sheets["PDF Summary"]
    
    # 調整欄位寬度
    for i, col in enumerate(pdf_summary_df.columns):
        max_width = max(pdf_summary_df[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.column_dimensions[get_column_letter(i + 1)].width = max_width
    
    # 文字換行
    for row in worksheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

print(f"PDF 摘要已保存至 {pdf_summary_excel_path}")

# 第二部分：生成包含客戶特徵、新聞摘要、投資建議和影響分數的 Excel 文件
df = pd.read_excel("teste.xlsx")  # 讀取您的客戶數據文件 test.xlsx
all_customers_data = []

selected_columns = [
    '客戶風險等級','近一年總成交量', '股票類型交易量', '債券類型交易量', '平衡類型交易量', 
    '股票類型交易量佔比', '債券類型交易量佔比', '平衡類型交易量佔比','近一年總成交次數', 
    '股票類型交易次數', '債券類型交易次數', '平衡類型交易次數','股票類型交易次數佔比', 
    '債券類型交易次數佔比','平衡類型交易次數佔比','庫存量','股票類型庫存量','債券類型庫存量',
    '平衡類型庫存量','股票類型庫存量佔比','債券類型庫存量佔比','平衡類型庫存量佔比',
    'RR1佔比','RR2佔比','RR3佔比','RR4佔比','RR5佔比','近一年REITs的總成交量',
    '近一年大中華股票的總成交量','近一年中國股票的總成交量','近一年巴西股票的總成交量',
    '近一年日本中小型股票的總成交量','近一年日本股票的總成交量','近一年台灣股票的總成交量',
    '近一年全球平衡的總成交量','近一年全球股票的總成交量','近一年全球債的總成交量',
    '近一年印度股票的總成交量','近一年亞太股票的總成交量','近一年亞太區(除日本)股票的總成交量',
    '近一年亞太區股票的總成交量','近一年亞太區貨幣債的總成交量','近一年拉美股票的總成交量',
    '近一年東南亞股票的總成交量','近一年金融股票 的總成交量','近一年南韓股票的總成交量',
    '近一年科技股票的總成交量','近一年美元高息/高收益債券的總成交量','近一年美國股票的總成交量',
    '近一年美債的總成交量','近一年英國股票的總成交量','近一年香港股票的總成交量','RR5佔比',
    '近一年高收益債的總成交量','近一年貨幣型的總成交量','近一年黃金的總成交量','近一年新興市場股票的總成交量',
    '近一年新興市場債的總成交量','近一年新興歐洲股票的總成交量','近一年德國股票的總成交量',
    '近一年歐洲中小型股票的總成交量','近一年歐洲股票的總成交量','近一年歐洲新興市場股票的總成交量',
    '近一年環球股票的總成交量','近一年環球高息/高收益債券的總成交量','近一年環球債券的總成交量',
    '近一年環球新市債的總成交量','近一年環球新興市場股票的總成交量','近一年環球新興市場強勢貨幣債券的總成交量',
    '近一年醫療生技的總成交量','近一年醫療保健股票的總成交量','近一年礦業股票的總成交量'
]

for index, row in df.iterrows():
    customer_name = row['ID']
    
    # 根據選擇的欄位來生成客戶特徵
    customer_features = []
    for col in selected_columns:
        if pd.notna(row[col]):
            feature = f"{col}: {row[col]}"
            customer_features.append(feature)
    
    # 更新狀態
    state = {
        "customer_features": customer_features,
        "news_content": "來自新聞的數據"  # 這裡可以填入每頁生成的新聞摘要
    }

    # 執行流程
    result = bot.graph.invoke(state)

    all_customers_data.append({
        "客戶": customer_name,
        "客戶特徵": result["extracted_features"],
        "新聞摘要": result["news_summary"],
        "投資建議": result["investment_advice_result"],
        "影響程度分數": result["impact_score"]
    })

# 保存客戶的數據到 Excel
customer_output_df = pd.DataFrame(all_customers_data)
customer_excel_path = "Customer_Investment_Advice.xlsx"

with pd.ExcelWriter(customer_excel_path, engine="openpyxl") as writer:
    customer_output_df.to_excel(writer, sheet_name="Investment Advice", index=False)
    worksheet = writer.sheets["Investment Advice"]
    
    # 調整欄位寬度
    for i, col in enumerate(customer_output_df.columns):
        max_width = max(customer_output_df[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.column_dimensions[get_column_letter(i + 1)].width = max_width
    
    # 文字換行
    for row in worksheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

print(f"客戶投資建議已保存至 {customer_excel_path}")


TypeError: a bytes-like object is required, not 'Image'

In [1]:
pip install pytesseract

In [2]:
import pandas as pd
import pdfplumber
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List, TypedDict
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
import re

# 定義狀態字典類型，用於保存每個客戶的狀態信息
class CustomerState(TypedDict):
    news_content: str  # 輸入的新聞內容
    customer_features: List[str]  # 客戶特徵列表
    extracted_features: str  # 提取的客戶特徵摘要
    news_summary: str  # 基於客戶特徵的新聞摘要
    investment_advice_result: str  # 生成的投資建議
    impact_score: int  # 新增的影響程度分數

# 定義個性化新聞代理類
class PersonalizedNewsAgent:

    def __init__(self, model: BaseChatModel, checkpointer: SqliteSaver):
        self.model = model  # 初始化生成式語言模型
        self.checkpointer = checkpointer  # 初始化檢查點保存器，用於保存狀態
        builder = StateGraph(CustomerState)  # 創建StateGraph並傳入狀態字典類型
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("extract_customer_features", self.extract_customer_features_node)
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)
        builder.add_node("investment_advice", self.investment_advice_node)
        builder.add_node("impact_assessment", self.impact_assessment_node)  # 新增影響評估節點
        builder.add_node("output_results", self.output_results_node)

        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "extract_customer_features")
        builder.add_edge("extract_customer_features", "personalized_news_summary")
        builder.add_edge("personalized_news_summary", "investment_advice")
        builder.add_edge("investment_advice", "impact_assessment")  # 添加新的邊
        builder.add_edge("impact_assessment", "output_results")
        self.graph = builder.compile(checkpointer=checkpointer)

    def initialize_node(self, state: CustomerState):
        return {
            "extracted_features": "",
            "news_summary": "",
            "investment_advice_result": "",
            "impact_score": 50,  # 初始化影響程度分數為 50（中等）
        }

    def extract_customer_features_node(self, state: CustomerState):
        customer_features_text = "\n".join(state["customer_features"])
        messages = [
            SystemMessage(content="以下所有產品皆為基金類型的產品，請直接列點總結每個客戶特徵，100字內，列點包括投資配置、產品偏好、風險偏好、投資策略、交易狀況、持倉狀況等，並確保生成內容完全來自以下客戶特徵信息，不參考其他來源。請用繁體中文回答。"),
            HumanMessage(content=customer_features_text),
        ]
        response = self.model.invoke(messages)
        state["extracted_features"] = response.content.strip()
        return {"extracted_features": state["extracted_features"]}

    def personalized_news_summary_node(self, state: CustomerState):
        # 結合客戶特徵和新聞內容來生成摘要
        combined_content = f"客戶特徵: {state['extracted_features']}\n新聞內容: {state['news_content']}"
        
        messages = [
            SystemMessage(content="請根據以下客戶特徵與新聞內容生成新聞重點摘要，個人化條列式生成3至5點，與客戶投資產品相關的新聞放在最前面，每點字數不超過30字，總字數少於200字。請用繁體中文回答。"),
            HumanMessage(content=combined_content)
        ]
        
        response = self.model.invoke(messages)
        state["news_summary"] = response.content.strip()
        
        return {"news_summary": state["news_summary"]}

    def investment_advice_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="你是一位具有 CFA Level 3 資格的資深投資專家，請嚴格基於每位客戶特徵和新聞摘要，列點式生成個人化投資建議，直接給具體建議即可。不要加入其他外部資訊，不要推薦任何產品，建議字數應在200字內。請確保新聞摘要中的資訊有確實反映在投資建議中，投資建議應該考慮到客戶的風險承受能力、投資目標、資產配置偏好，以及當前市場狀況，務必確保每個建議都是實際可行的。請用繁體中文回答。"),
            AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
            AIMessage(content=f"新聞摘要: {state['news_summary']}"),
        ]
        response = self.model.invoke(messages)
        state["investment_advice_result"] = response.content.strip()
        return {"investment_advice_result": state["investment_advice_result"]}

    def impact_assessment_node(self, state: CustomerState):
        messages = [
        SystemMessage(content="你是一位具有 CFA Level 3 資格的資深投資專家，請評估新聞對每個客戶的影響程度。給出0到100分的評分，分數越高代表影響越大。"),
        AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
        AIMessage(content=f"新聞摘要: {state['news_summary']}"),
    ]
        response = self.model.invoke(messages)
        impact_text = response.content.strip()

        # 初始化默認值
        score = 20
        explanation = "影響甚小，先給予20分"

        # 檢查是否模型返回了完整的提示而不是结果
        if "0到100分的評分" not in impact_text:
            # 使用正則表達式提取数字評分
            score_match = re.search(r'\b(\d{1,3})\b', impact_text)
            if score_match:
                score = int(score_match.group(1))
                # 保证分數在0到100之间
                score = min(max(score, 0), 100)
                explanation = impact_text[impact_text.find(score_match.group(1)) + len(score_match.group(1)):].strip()
                explanation = explanation.replace('*', '').strip()  # 移除可能的米字號或其他不必要符号
            else:
                # 如果没有提取到有效分数，使用範例值並解釋
                score = 20
                explanation = "影響甚小，先給予20分"

        # 构建影响评分结果，确保没有多余的分数后缀
        state["impact_score"] = f"{score} 分" if explanation else f"{score} 分 - {explanation}"
        return {"impact_score": state["impact_score"]}

    def output_results_node(self, state: CustomerState):
        return {
            "extracted_features": state["extracted_features"],
            "news_summary": state["news_summary"],
            "investment_advice_result": state["investment_advice_result"],
            "impact_score": state["impact_score"],  # 包含影響程度分數
        }

# 定義格式化輸出結果的函數
def format_output_for_customer(customer_name, extracted_features, news_summary, investment_advice_result, impact_score):
    return (f"客戶名稱: {customer_name}\n"
            f"提取的客戶特徵:\n{extracted_features}\n\n"
            f"新聞摘要:\n{news_summary}\n\n"
            f"投資建議:\n{investment_advice_result}\n\n"
            f"新聞對客戶的影響程度: {impact_score} 分\n")

# 新增函數：從PDF讀取新聞內容並逐頁提取摘要
def extract_news_summary_from_pdf(pdf_path: str, model: BaseChatModel) -> List[str]:
    summaries = []
    try:
        # 打開PDF文件並提取每頁內容
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, start=1):
                news_content = page.extract_text()
                if news_content:
                    # 生成新聞摘要
                    messages = [
                        SystemMessage(content="請根據以下新聞內容生成摘要，字數不超過200字。請用繁體中文回答。"),
                        HumanMessage(content=news_content)
                    ]
                    response = model.invoke(messages)
                    summary = response.content.strip()
                    summaries.append(f"Page {page_num} Summary: {summary}")
    except Exception as e:
        print(f"無法讀取PDF文件: {e}")
    return summaries

# 初始化模型和檢查點保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
memory = SqliteSaver.from_conn_string(":memory:")

bot = PersonalizedNewsAgent(model=model, checkpointer=memory)

# 使用 PDF 路徑來提取新聞並逐頁生成摘要
pdf_path = "test.pdf"  # 替換為你的PDF文件路徑
news_summaries = extract_news_summary_from_pdf(pdf_path, model)

# 生成摘要的 DataFrame 並保存到 Excel
summary_df = pd.DataFrame({"Page Summary": news_summaries})

# 保存摘要到 Excel
summary_output_path = "PDF_News_Summaries.xlsx"
with pd.ExcelWriter(summary_output_path, engine="openpyxl") as writer:
    summary_df.to_excel(writer, sheet_name="Summaries", index=False)
    worksheet = writer.sheets["Summaries"]
    
    # 調整欄位寬度
    for i, col in enumerate(summary_df.columns):
        max_width = max(summary_df[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.column_dimensions[get_column_letter(i + 1)].width = max_width
    
    # 文字換行
    for row in worksheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

print(f"PDF 逐頁摘要已生成並保存至 {summary_output_path}")

# 定義對話線程的配置
thread = {"configurable": {"thread_id": "1"}}

# 讀取Excel表格
df = pd.read_excel("teste.xlsx")
    
# 保留完整的 selected_columns 列表
selected_columns = ['客戶風險等級','近一年總成交量', '股票類型交易量', '債券類型交易量', '平衡類型交易量', '股票類型交易量佔比', '債券類型交易量佔比', '平衡類型交易量佔比'
                    ,'近一年總成交次數','股票類型交易次數','債券類型交易次數','平衡類型交易次數','股票類型交易次數佔比','債券類型交易次數佔比'
                    ,'平衡類型交易次數佔比','庫存量','股票類型庫存量','債券類型庫存量','平衡類型庫存量','股票類型庫存量佔比','債券類型庫存量佔比'
                    ,'平衡類型庫存量佔比','RR1佔比','RR2佔比','RR3佔比','RR4佔比','RR5佔比','近一年REITs的總成交量','近一年大中華股票的總成交量'
                    ,'近一年中國股票的總成交量','近一年巴西股票的總成交量','近一年日本中小型股票的總成交量','近一年日本股票的總成交量'
                    ,'近一年台灣股票的總成交量','近一年全球平衡的總成交量','近一年全球股票的總成交量','近一年全球債的總成交量','近一年印度股票的總成交量'
                    ,'近一年亞太股票的總成交量','近一年亞太區(除日本)股票的總成交量','近一年亞太區股票的總成交量','近一年亞太區貨幣債的總成交量'
                    ,'近一年拉美股票的總成交量','近一年東南亞股票的總成交量','近一年金融股票 的總成交量','近一年南韓股票的總成交量'
                    ,'近一年科技股票的總成交量','近一年美元高息/高收益債券的總成交量','近一年美國股票的總成交量','近一年美債的總成交量'
                    ,'近一年英國股票的總成交量','近一年香港股票的總成交量','RR5佔比','近一年高收益債的總成交量','近一年貨幣型的總成交量'
                    ,'近一年黃金的總成交量','近一年新興市場股票的總成交量','近一年新興市場債的總成交量','近一年新興歐洲股票的總成交量'
                    ,'近一年德國股票的總成交量','近一年歐洲中小型股票的總成交量','近一年歐洲股票的總成交量','近一年歐洲新興市場股票的總成交量'
                    ,'近一年環球股票的總成交量','近一年環球高息/高收益債券的總成交量','近一年環球債券的總成交量','近一年環球新市債的總成交量'
                    ,'近一年環球新興市場股票的總成交量','近一年環球新興市場強勢貨幣債券的總成交量','近一年醫療生技的總成交量'
                    ,'近一年醫療保健股票的總成交量','近一年礦業股票的總成交量','近一年REITs的總成交量佔比','近一年大中華股票的總成交量佔比'
                    ,'近一年中國股票的總成交量佔比','近一年巴西股票的總成交量佔比','近一年日本中小型股票的總成交量佔比'
                    ,'近一年日本股票的總成交量佔比','近一年台灣股票的總成交量佔比','近一年全球平衡的總成交量佔比','近一年全球股票的總成交量佔比'
                    ,'近一年全球債的總成交量佔比','近一年印度股票的總成交量佔比','近一年亞太股票的總成交量佔比','近一年亞太區(除日本)股票的總成交量佔比'
                    ,'近一年亞太區股票的總成交量佔比','近一年亞太區貨幣債的總成交量佔比','近一年拉美股票的總成交量佔比','近一年東南亞股票的總成交量佔比'
                    ,'近一年金融股票 的總成交量佔比','近一年南韓股票的總成交量佔比','近一年科技股票的總成交量佔比','近一年美元高息/高收益債券的總成交量佔比'
                    ,'近一年美國股票的總成交量佔比','近一年美債的總成交量佔比','近一年英國股票的總成交量佔比','近一年香港股票的總成交量佔比'
                    ,'近一年能源股票 的總成交量佔比','近一年高收益債的總成交量佔比','近一年貨幣型的總成交量佔比','近一年黃金的總成交量佔比'
                    ,'近一年新興市場股票的總成交量佔比','近一年新興市場債的總成交量佔比','近一年新興歐洲股票的總成交量佔比','近一年德國股票的總成交量佔比'
                    ,'近一年歐洲中小型股票的總成交量佔比','近一年歐洲股票的總成交量佔比','近一年歐洲新興市場股票的總成交量佔比','近一年環球股票的總成交量佔比'
                    ,'近一年環球高息/高收益債券的總成交量佔比','近一年環球債券的總成交量佔比','近一年環球新市債的總成交量佔比'
                    ,'近一年環球新興市場股票的總成交量佔比','近一年環球新興市場強勢貨幣債券的總成交量佔比','近一年醫療生技的總成交量佔比'
                    ,'近一年醫療保健股票的總成交量佔比','近一年礦業股票的總成交量佔比','近一年REITs的總成交次數','近一年大中華股票的總成交次數'
                    ,'近一年中國股票的總成交次數','近一年巴西股票的總成交次數','近一年日本中小型股票的總成交次數','近一年日本股票的總成交次數'
                    ,'近一年台灣股票的總成交次數','近一年全球平衡的總成交次數','近一年全球股票的總成交次數','近一年全球債的總成交次數'
                    ,'近一年印度股票的總成交次數','近一年亞太股票的總成交次數','近一年亞太區(除日本)股票的總成交次數','近一年亞太區股票的總成交次數'
                    ,'近一年亞太區貨幣債的總成交次數','近一年拉美股票的總成交次數','近一年東南亞股票的總成交次數','近一年金融股票 的總成交次數'
                    ,'近一年南韓股票的總成交次數','近一年科技股票的總成交次數','近一年美元高息/高收益債券的總成交次數','近一年美國股票的總成交次數'
                    ,'近一年美債的總成交次數','近一年英國股票的總成交次數','近一年香港股票的總成交次數','近一年能源股票 的總成交次數'
                    ,'近一年高收益債的總成交次數','近一年貨幣型的總成交次數','近一年黃金的總成交次數','近一年新興市場股票的總成交次數'
                    ,'近一年新興市場債的總成交次數','近一年新興歐洲股票的總成交次數','近一年德國股票的總成交次數','近一年歐洲中小型股票的總成交次數'
                    ,'近一年歐洲股票的總成交次數','近一年歐洲新興市場股票的總成交次數','近一年環球股票的總成交次數','近一年環球高息/高收益債券的總成交次數'
                    ,'近一年環球債券的總成交次數','近一年環球新市債的總成交次數','近一年環球新興市場股票的總成交次數','近一年環球新興市場強勢貨幣債券的總成交次數'
                    ,'近一年醫療生技的總成交次數','近一年醫療保健股票的總成交次數','近一年礦業股票的總成交次數','近一年REITs的總成交次數佔比'
                    ,'近一年大中華股票的總成交次數佔比','近一年中國股票的總成交次數佔比','近一年巴西股票的總成交次數佔比','近一年日本中小型股票的總成交次數佔比'
                    ,'近一年日本股票的總成交次數佔比','近一年台灣股票的總成交次數佔比','近一年全球平衡的總成交次數佔比','近一年全球股票的總成交次數佔比'
                    ,'近一年全球債的總成交次數佔比','近一年印度股票的總成交次數佔比','近一年亞太股票的總成交次數佔比','近一年亞太區(除日本)股票的總成交次數佔比'
                    ,'近一年亞太區股票的總成交次數佔比','近一年亞太區貨幣債的總成交次數佔比','近一年拉美股票的總成交次數佔比','近一年東南亞股票的總成交次數佔比'
                    ,'近一年金融股票 的總成交次數佔比','近一年南韓股票的總成交次數佔比','近一年科技股票的總成交次數佔比'
                    ,'近一年美元高息/高收益債券的總成交次數佔比','近一年美國股票的總成交次數佔比','近一年美債的總成交次數佔比'
                    ,'近一年英國股票的總成交次數佔比','近一年香港股票的總成交次數佔比','近一年能源股票 的總成交次數佔比','近一年高收益債的總成交次數佔比'
                    ,'近一年貨幣型的總成交次數佔比','近一年黃金的總成交次數佔比','近一年新興市場股票的總成交次數佔比','近一年新興市場債的總成交次數佔比'
                    ,'近一年新興歐洲股票的總成交次數佔比','近一年德國股票的總成交次數佔比','近一年歐洲中小型股票的總成交次數佔比','近一年歐洲股票的總成交次數佔比'
                    ,'近一年歐洲新興市場股票的總成交次數佔比','近一年環球股票的總成交次數佔比','近一年環球高息/高收益債券的總成交次數佔比'
                    ,'近一年環球債券的總成交次數佔比','近一年環球新市債的總成交次數佔比','近一年環球新興市場股票的總成交次數佔比'
                    ,'近一年環球新興市場強勢貨幣債券的總成交次數佔比','近一年醫療生技的總成交次數佔比','近一年醫療保健股票的總成交次數佔比'
                    ,'近一年礦業股票的總成交次數佔比','REITs的庫存量','大中華股票的庫存量','中國股票的庫存量','巴西股票的庫存量'
                    ,'日本中小型股票的庫存量','日本股票的庫存量','台灣股票的庫存量','全球平衡的庫存量','全球股票的庫存量','全球債的庫存量'
                    ,'印度股票的庫存量','亞太股票的庫存量','亞太區(除日本)股票的庫存量','亞太區股票的庫存量','亞太區貨幣債的庫存量','拉美股票的庫存量'
                    ,'東南亞股票的庫存量','金融股票 的庫存量','南韓股票的庫存量','科技股票的庫存量','美元高息/高收益債券的庫存量','美國股票的庫存量'
                    ,'美債的庫存量','英國股票的庫存量','香港股票的庫存量','能源股票 的庫存量','高收益債的庫存量','貨幣型的庫存量','黃金的庫存量'
                    ,'新興市場股票的庫存量','新興市場債的庫存量','新興歐洲股票的庫存量','德國股票的庫存量','歐洲中小型股票的庫存量','歐洲股票的庫存量'
                    ,'歐洲新興市場股票的庫存量','環球股票的庫存量','環球高息/高收益債券的庫存量','環球債券的庫存量','環球新市債的庫存量'
                    ,'環球新興市場股票的庫存量','環球新興市場強勢貨幣債券的庫存量','醫療生技的庫存量','醫療保健股票的庫存量','礦業股票的庫存量'
                    ,'REITs的庫存量佔比','大中華股票的庫存量佔比','中國股票的庫存量佔比','巴西股票的庫存量佔比','日本中小型股票的庫存量佔比'
                    ,'日本股票的庫存量佔比','台灣股票的庫存量佔比','全球平衡的庫存量佔比','全球股票的庫存量佔比','全球債的庫存量佔比','印度股票的庫存量佔比'
                    ,'亞太股票的庫存量佔比','亞太區(除日本)股票的庫存量佔比','亞太區股票的庫存量佔比','亞太區貨幣債的庫存量佔比','拉美股票的庫存量佔比'
                    ,'東南亞股票的庫存量佔比','金融股票 的庫存量佔比','南韓股票的庫存量佔比','科技股票的庫存量佔比','美元高息/高收益債券的庫存量佔比'
                    ,'美國股票的庫存量佔比','美債的庫存量佔比','英國股票的庫存量佔比','香港股票的庫存量佔比','能源股票 的庫存量佔比','高收益債的庫存量佔比'
                    ,'貨幣型的庫存量佔比','黃金的庫存量佔比','新興市場股票的庫存量佔比','新興市場債的庫存量佔比','新興歐洲股票的庫存量佔比'
                    ,'德國股票的庫存量佔比','歐洲中小型股票的庫存量佔比','歐洲股票的庫存量佔比','歐洲新興市場股票的庫存量佔比','環球股票的庫存量佔比'
                    ,'環球高息/高收益債券的庫存量佔比','環球債券的庫存量佔比','環球新市債的庫存量佔比','環球新興市場股票的庫存量佔比'
                    ,'環球新興市場強勢貨幣債券的庫存量佔比','醫療生技的庫存量佔比','醫療保健股票的庫存量佔比','礦業股票的庫存量佔比'
                   ]

# 生成客戶特徵和投資建議流程保持不變
all_customers_output = []
all_customers_data = []

for index, row in df.iterrows():
    customer_name = row['ID']
    
    # 根據選擇的欄位來生成客戶特徵
    customer_features = []
    for col in selected_columns:
        if pd.notna(row[col]):  # 檢查該欄位是否有數值
            feature = f"{col}: {row[col]}"
            customer_features.append(feature)
    
    # 更新狀態
    state = {"customer_features": customer_features, "news_content": ""}

    # 執行圖形流程
    result = bot.graph.invoke(state, thread)

    # 檢查生成的投資建議是否為空，如果是空的，給予警告並記錄
    if not result["investment_advice_result"]:
        print(f"警告: {customer_name} 的投資建議生成為空！")

    # 格式化輸出結果
    customer_output = format_output_for_customer(
        customer_name, 
        result["extracted_features"], 
        result["news_summary"],
        result["investment_advice_result"], 
        result["impact_score"]  # 添加影響程度分數
    )
    
    all_customers_output.append(customer_output)

    all_customers_data.append({
        "客戶": customer_name,
        "客戶特徵": result["extracted_features"],
        "新聞摘要": result["news_summary"],
        "投資建議": result["investment_advice_result"],
        "影響程度分數": result["impact_score"]  # 新增的影響程度分數列
    })

# 創建DataFrame並輸出到Excel
output_df = pd.DataFrame(all_customers_data)
with pd.ExcelWriter("Personalized_Investment_Advice_Output.xlsx", engine="openpyxl") as writer:
    output_df.to_excel(writer, sheet_name="Sheet1", index=False)
    worksheet = writer.sheets["Sheet1"]
    
    # 調整欄位寬度
    for i, col in enumerate(output_df.columns):
        max_width = max(output_df[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.column_dimensions[get_column_letter(i + 1)].width = max_width
    
    # 文字換行
    for row in worksheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

print("Excel 文件已生成並保存，請檢查結果。")


PDF 逐頁摘要已生成並保存至 PDF_News_Summaries.xlsx
Excel 文件已生成並保存，請檢查結果。


In [3]:
import pandas as pd
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List, TypedDict
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
import re
import pdfplumber

# 定義狀態字典類型，用於保存每個客戶的狀態信息
class CustomerState(TypedDict):
    news_content: str  # 輸入的新聞內容
    customer_features: List[str]  # 客戶特徵列表
    extracted_features: str  # 提取的客戶特徵摘要
    news_summary: str  # 基於客戶特徵的新聞摘要
    investment_advice_result: str  # 生成的投資建議
    impact_score: int  # 新增的影響程度分數

# 定義個性化新聞代理類
class PersonalizedNewsAgent:

    def __init__(self, model: BaseChatModel, checkpointer: SqliteSaver):
        self.model = model  # 初始化生成式語言模型
        self.checkpointer = checkpointer  # 初始化檢查點保存器，用於保存狀態
        builder = StateGraph(CustomerState)  # 創建StateGraph並傳入狀態字典類型
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("extract_customer_features", self.extract_customer_features_node)
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)
        builder.add_node("investment_advice", self.investment_advice_node)
        builder.add_node("impact_assessment", self.impact_assessment_node)  # 新增影響評估節點
        builder.add_node("output_results", self.output_results_node)

        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "extract_customer_features")
        builder.add_edge("extract_customer_features", "personalized_news_summary")
        builder.add_edge("personalized_news_summary", "investment_advice")
        builder.add_edge("investment_advice", "impact_assessment")  # 添加新的邊
        builder.add_edge("impact_assessment", "output_results")
        self.graph = builder.compile(checkpointer=checkpointer)

    def initialize_node(self, state: CustomerState):
        return {
            "extracted_features": "",
            "news_summary": "",
            "investment_advice_result": "",
            "impact_score": 50,  # 初始化影響程度分數為 50（中等）
        }

    def extract_customer_features_node(self, state: CustomerState):
        customer_features_text = "\n".join(state["customer_features"])
        messages = [
            SystemMessage(content="以下所有產品皆為基金類型的產品，請直接列點總結每個客戶特徵，100字內，列點包括投資配置、產品偏好、風險偏好、投資策略、交易狀況、持倉狀況等，並確保生成內容完全來自以下客戶特徵信息，不參考其他來源。請用繁體中文回答。"),
            HumanMessage(content=customer_features_text),
        ]
        response = self.model.invoke(messages)
        state["extracted_features"] = response.content.strip()
        return {"extracted_features": state["extracted_features"]}

    def personalized_news_summary_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="請根據以下新聞內容及客戶特徵生成新聞重點摘要，個人化條列式生成3至5點，與客戶投資產品相關的新聞放在最前面，每點字數不超過30字，總字數少於200字，不要包括任何客戶特徵信息，只能從以下新聞內容生成摘要。請用繁體中文回答"),
            HumanMessage(content=state["news_content"])
        ]
        response = self.model.invoke(messages)
        state["news_summary"] = response.content.strip()
        return {"news_summary": state["news_summary"]}

    def investment_advice_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="你是一位具有 CFA Level 3 資格的資深投資專家，請嚴格基於每位客戶特徵和新聞摘要，列點式生成個人化投資建議，直接給具體建議即可。不要加入其他外部資訊，不要推薦任何產品，建議字數應在200字內。請確保新聞摘要中的資訊有確實反映在投資建議中，投資建議應該考慮到客戶的風險承受能力、投資目標、資產配置偏好，以及當前市場狀況，務必確保每個建議都是實際可行的。請用繁體中文回答。"),
            AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
            AIMessage(content=f"新聞摘要: {state['news_summary']}"),
        ]
        response = self.model.invoke(messages)
        state["investment_advice_result"] = response.content.strip()
        return {"investment_advice_result": state["investment_advice_result"]}

    def impact_assessment_node(self, state: CustomerState):
        messages = [
        SystemMessage(content="你是一位具有 CFA Level 3 資格的資深投資專家，請評估新聞對每個客戶的影響程度。給出0到100分的評分，分數越高代表影響越大。"),
        AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
        AIMessage(content=f"新聞摘要: {state['news_summary']}"),
    ]
        response = self.model.invoke(messages)
        impact_text = response.content.strip()

    # 初始化默認值
        score = 20
        explanation = "影響甚小，先給予20分"

    # 檢查是否模型返回了完整的提示而不是结果
        if "0到100分的評分" not in impact_text:
        # 使用正則表達式提取数字評分
            score_match = re.search(r'\b(\d{1,3})\b', impact_text)
            if score_match:
                score = int(score_match.group(1))
            # 保证分數在0到100之间
                score = min(max(score, 0), 100)
                explanation = impact_text[impact_text.find(score_match.group(1)) + len(score_match.group(1)):].strip()
                explanation = explanation.replace('*', '').strip()  # 移除可能的米字號或其他不必要符号
            else:

            # 如果没有提取到有效分数，使用範例值並解釋
                score = 20
                explanation = "影響甚小，先給予20分"

    # 构建影响评分结果，确保没有多余的分数后缀
        state["impact_score"] = f"{score} 分" if explanation else f"{score} 分 - {explanation}"
        return {"impact_score": state["impact_score"]}

    def output_results_node(self, state: CustomerState):
        return {
            "extracted_features": state["extracted_features"],
            "news_summary": state["news_summary"],
            "investment_advice_result": state["investment_advice_result"],
            "impact_score": state["impact_score"],  # 包含影響程度分數
        }

# 定義格式化輸出結果的函數
def format_output_for_customer(customer_name, extracted_features, news_summary, investment_advice_result, impact_score):
    return (f"客戶名稱: {customer_name}\n"
            f"提取的客戶特徵:\n{extracted_features}\n\n"
            f"新聞摘要:\n{news_summary}\n\n"
            f"投資建議:\n{investment_advice_result}\n\n"
            f"新聞對客戶的影響程度: {impact_score} 分\n")

# 定義PDF提取摘要的函數
def extract_pdf_summary(pdf_file_path):
    summary = []
    with pdfplumber.open(pdf_file_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            summary.append(text[:200] + "...")  # 提取前200個字作為摘要
    return "\n".join(summary)

# 初始化模型和檢查點保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
memory = SqliteSaver.from_conn_string(":memory:")

bot = PersonalizedNewsAgent(model=model, checkpointer=memory)
    
thread = {"configurable": {"thread_id": "1"}}

# 讀取Excel表格
df = pd.read_excel("teste.xlsx")

# 從PDF提取新聞內容
pdf_summary = extract_pdf_summary("test.pdf")

# 定義輸入的新聞內容
state = {
    "news_content": pdf_summary
}

# 生成客戶特徵
all_customers_output = []
all_customers_data = []

# 你可以選擇指定要使用的欄位名稱
selected_columns = ['客戶風險等級','近一年總成交量', '股票類型交易量', '債券類型交易量', '平衡類型交易量', '股票類型交易量佔比', '債券類型交易量佔比', '平衡類型交易量佔比'
                    ,'近一年總成交次數','股票類型交易次數','債券類型交易次數','平衡類型交易次數','股票類型交易次數佔比','債券類型交易次數佔比'
                    ,'平衡類型交易次數佔比','庫存量','股票類型庫存量','債券類型庫存量','平衡類型庫存量','股票類型庫存量佔比','債券類型庫存量佔比'
                    ,'平衡類型庫存量佔比','RR1佔比','RR2佔比','RR3佔比','RR4佔比','RR5佔比','近一年REITs的總成交量','近一年大中華股票的總成交量'
                    ,'近一年中國股票的總成交量','近一年巴西股票的總成交量','近一年日本中小型股票的總成交量','近一年日本股票的總成交量'
                    ,'近一年台灣股票的總成交量','近一年全球平衡的總成交量','近一年全球股票的總成交量','近一年全球債的總成交量','近一年印度股票的總成交量'
                    ,'近一年亞太股票的總成交量','近一年亞太區(除日本)股票的總成交量','近一年亞太區股票的總成交量','近一年亞太區貨幣債的總成交量'
                    ,'近一年拉美股票的總成交量','近一年東南亞股票的總成交量','近一年金融股票 的總成交量','近一年南韓股票的總成交量'
                    ,'近一年科技股票的總成交量','近一年美元高息/高收益債券的總成交量','近一年美國股票的總成交量','近一年美債的總成交量'
                    ,'近一年英國股票的總成交量','近一年香港股票的總成交量','RR5佔比','近一年高收益債的總成交量','近一年貨幣型的總成交量'
                    ,'近一年黃金的總成交量','近一年新興市場股票的總成交量','近一年新興市場債的總成交量','近一年新興歐洲股票的總成交量','近一年德國股票的總成交量'
                    ,'近一年歐洲中小型股票的總成交量','近一年歐洲股票的總成交量','近一年歐洲新興市場股票的總成交量','近一年環球股票的總成交量佔比'
                    ,'近一年環球高息/高收益債券的總成交量佔比','近一年環球債券的總成交量佔比','近一年環球新市債的總成交量佔比'
                    ,'近一年環球新興市場股票的總成交量佔比','近一年環球新興市場強勢貨幣債券的總成交量佔比','近一年醫療生技的總成交量佔比'
                    ,'近一年醫療保健股票的總成交量佔比','近一年礦業股票的總成交量佔比','近一年REITs的總成交量佔比','近一年大中華股票的總成交量佔比'
                    ,'近一年中國股票的總成交量佔比','近一年巴西股票的總成交量佔比','近一年日本中小型股票的總成交量佔比'
                    ,'近一年日本股票的總成交量佔比','近一年台灣股票的總成交量佔比','近一年全球平衡的總成交量佔比','近一年全球股票的總成交量佔比'
                    ,'近一年全球債的總成交量佔比','近一年印度股票的總成交量佔比','近一年亞太股票的總成交量佔比','近一年亞太區(除日本)股票的總成交量佔比'
                    ,'近一年亞太區股票的總成交量佔比','近一年亞太區貨幣債的總成交量佔比','近一年拉美股票的總成交量佔比','近一年東南亞股票的總成交量佔比'
                    ,'近一年金融股票 的總成交量佔比','近一年南韓股票的總成交量佔比','近一年科技股票的總成交量佔比','近一年美元高息/高收益債券的總成交量佔比'
                    ,'近一年美國股票的總成交量佔比','近一年美債的總成交量佔比','近一年英國股票的總成交量佔比','近一年香港股票的總成交量佔比'
                    ,'近一年能源股票 的總成交量佔比','近一年高收益債的總成交量佔比','近一年貨幣型的總成交量佔比','近一年黃金的總成交量佔比'
                    ,'近一年新興市場股票的總成交量佔比','近一年新興市場債的總成交量佔比','近一年新興歐洲股票的總成交量佔比','近一年德國股票的總成交量佔比'
                    ,'近一年歐洲中小型股票的總成交量佔比','近一年歐洲股票的總成交量佔比','近一年歐洲新興市場股票的總成交量佔比','近一年環球股票的總成交量佔比'
                    ,'近一年環球高息/高收益債券的總成交量佔比','近一年環球債券的總成交量佔比','近一年環球新市債的總成交量佔比'
                    ,'近一年環球新興市場股票的總成交量佔比','近一年環球新興市場強勢貨幣債券的總成交量佔比','近一年醫療生技的總成交量佔比'
                    ,'近一年醫療保健股票的總成交量佔比','近一年礦業股票的總成交量佔比','REITs的庫存量','大中華股票的庫存量','中國股票的庫存量','巴西股票的庫存量'
                    ,'日本中小型股票的庫存量','日本股票的庫存量','台灣股票的庫存量','全球平衡的庫存量','全球股票的庫存量','全球債的庫存量'
                    ,'印度股票的庫存量','亞太股票的庫存量','亞太區(除日本)股票的庫存量','亞太區股票的庫存量','亞太區貨幣債的庫存量','拉美股票的庫存量'
                    ,'東南亞股票的庫存量','金融股票 的庫存量','南韓股票的庫存量','科技股票的庫存量','美元高息/高收益債券的庫存量','美國股票的庫存量'
                    ,'美債的庫存量','英國股票的庫存量','香港股票的庫存量','能源股票 的庫存量','高收益債的庫存量','貨幣型的庫存量','黃金的庫存量'
                    ,'新興市場股票的庫存量','新興市場債的庫存量','新興歐洲股票的庫存量','德國股票的庫存量','歐洲中小型股票的庫存量','歐洲股票的庫存量'
                    ,'歐洲新興市場股票的庫存量','環球股票的庫存量','環球高息/高收益債券的庫存量','環球債券的庫存量','環球新市債的庫存量'
                    ,'環球新興市場股票的庫存量','環球新興市場強勢貨幣債券的庫存量','醫療生技的庫存量','醫療保健股票的庫存量','礦業股票的庫存量'
                   ]

# 迭代每個客戶，生成特徵摘要和投資建議
for index, row in df.iterrows():
    customer_name = row['ID']
    
    # 根據選擇的欄位來生成客戶特徵
    customer_features = []
    for col in selected_columns:
        if pd.notna(row[col]):  # 檢查該欄位是否有數值
            feature = f"{col}: {row[col]}"
            customer_features.append(feature)
    
    # 更新狀態
    state["customer_features"] = customer_features

    # 執行圖形流程
    result = bot.graph.invoke(state, thread)

    # 檢查生成的投資建議是否為空，如果是空的，給予警告並記錄
    if not result["investment_advice_result"]:
        print(f"警告: {customer_name} 的投資建議生成為空！")

    # 格式化輸出結果
    customer_output = format_output_for_customer(
        customer_name, 
        result["extracted_features"], 
        result["news_summary"],
        result["investment_advice_result"], 
        result["impact_score"]  # 添加影響程度分數
    )
    
    all_customers_output.append(customer_output)

    all_customers_data.append({
        "客戶": customer_name,
        "客戶特徵": result["extracted_features"],
        "新聞摘要": result["news_summary"],
        "投資建議": result["investment_advice_result"],
        "影響程度分數": result["impact_score"]  # 新增的影響程度分數列
    })

# 創建DataFrame並輸出到Excel
output_df = pd.DataFrame(all_customers_data)
with pd.ExcelWriter("Personalized_Investment_Advice_Output.xlsx", engine="openpyxl") as writer:
    output_df.to_excel(writer, sheet_name="Sheet1", index=False)
    worksheet = writer.sheets["Sheet1"]
    
    # 調整欄位寬度
    for i, col in enumerate(output_df.columns):
        max_width = max(output_df[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.column_dimensions[get_column_letter(i + 1)].width = max_width
    
    # 文字換行
    for row in worksheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

print("Excel 文件已生成並保存，請檢查結果。")


Excel 文件已生成並保存，請檢查結果。
